In [1]:
import os
import sys
from pathlib import Path

# VSCode에서만 사용 가능한 건지 확인 필요
FILE = Path(__vsc_ipynb_file__).resolve()
ROOT = FILE.parents[1]
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from utils.dataloaders import LoadImagesAndLabels, create_dataloader

- image_weights가 어떤 기능을 제공하는지 구체적인 구현 내용 이해하고 정리하기
- Dataset, Dataloader에서 coll_fn, sampler의 기능과 동작 방식 이해하기

`utils/dataloaders.py`
`create_dataloader()`

```python
batch_size: 배치 크기
nd: CUDA 디바이스 개수(GPU 갯수)
nw: 워커 스레드 개수 (사용자 설정 workers, batch_size, max_cpu // nd 중 작은 숫자)
loader: image_weights가 참 이거나 close_mosaic이 참이면 DataLoader 아니면 InfiniteDataLoader
```

```python
class LoadImagesAndLabels(Dataset):
    def __init__(self):
        pass
```

In [2]:
params = {
    'path': '/work/dataset/VP-SAR-v1.0.0.all/dataset_debug.txt',
    'imgsz': 1280,
    'batch_size': 1,
    'stride': 1,
    'single_cls': True,
    'hyp': None,
    'augment': False,
    'cache': False,
    'pad': 0.0,
    'rect': False,
    'rank': -1,
    'workers': 8,
    'image_weights': False,
    'close_mosaic': False,
    'quad': False,
    'min_items': 0,
    'prefix': '',
    'shuffle': False
}
# print(**params)
dataloader, dataset = create_dataloader(**params)

Scanning /work/dataset/VP-SAR-v1.0.0.all/dataset_debug.cache... 131 images, 5 backgrounds, 0 corrupt: 100%|██████████| 131/131 00:00


In [24]:
print(len(dataset.labels))
# print(type(dataset.labels))
# print(dataset.labels[0])
nc = 2

131


In [25]:
import numpy as np
labels = np.concatenate(dataset.labels, 0)

In [26]:
classes = labels[:, 0].astype(int)

In [27]:
weights = np.bincount(classes, minlength=nc)

In [28]:
weights

array([181,   0])

In [29]:
weights[weights==0]=1
print(weights)

[181   1]


In [30]:
weights = 1 / weights
print(weights)

[  0.0055249           1]


In [32]:
weights.sum()

0.9999999999999999

In [31]:
weights /= weights.sum()
print(weights)

[  0.0054945     0.99451]


In [19]:
print(classes)
print(len(classes))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
181


In [21]:
print(labels.shape)
print(len(labels))

(181, 5)
181


In [6]:
from utils.general import labels_to_class_weights, labels_to_image_weights

In [41]:
# 데이터의 클래스에 라벨 빈도수가 높을 수록 가중치는 작아짐
class_weights = labels_to_class_weights(dataset.labels, nc)

In [34]:
for x in dataset.labels[:1]:
    print(x[:, 0])

[          0]


In [43]:
maps = np.zeros(nc)
print(maps)

[          0           0]


In [45]:
cw = class_weights.cpu().numpy() * (1 - maps) ** 2 / nc
print(cw)

[  0.0027473     0.49725]


In [42]:
# 각 이미지에 클래스 별 객체 빈도수 계산
class_counts = np.array([np.bincount(x[:, 0].astype(int), minlength=nc) for x in dataset.labels])
print(type(class_counts))
print(class_counts.shape)
print(class_weights.shape)
print(class_weights)

<class 'numpy.ndarray'>
(131, 2)
torch.Size([2])
tensor([0.00549, 0.99451])


https://docs.python.org/3/library/random.html#random.choices

`random.choices(population, weights=None, *, cum_weights=None, k=1)`